# 🥙 LSTM on Recipe Data

In this notebook, we'll walk through the steps required to train your own LSTM on the recipes dataset

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import json
import re
import string

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, losses

## 0. Parameters <a name="parameters"></a>

In [3]:
VOCAB_SIZE = 10000
MAX_LEN = 200
EMBEDDING_DIM = 100
N_UNITS = 128
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 25

## 1. Load the data <a name="load"></a>

In [1]:
# Download the dataset from Kaggle using kagglehub
# Note: You may need to configure Kaggle credentials first
# Set up credentials at: https://www.kaggle.com/settings -> API -> Create New API Token

import os

# Option 1: Using kagglehub (if available)
try:
    import kagglehub
    path = kagglehub.dataset_download("hugodarwood/epirecipes")
    print(f"Dataset downloaded to: {path}")
    
    # Copy the file to our data folder
    import shutil
    source_file = os.path.join(path, "full_format_recipes.json")
    dest_file = "./data/full_format_recipes.json"
    if os.path.exists(source_file):
        shutil.copy(source_file, dest_file)
        print(f"Copied to: {dest_file}")
except Exception as e:
    print(f"KaggleHub method failed: {e}")
    print("\nAlternative: Download manually from https://www.kaggle.com/datasets/hugodarwood/epirecipes")
    print("and place 'full_format_recipes.json' in the './data/' folder")

d:\Gen AI with LLM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 11.3M/11.3M [00:00<00:00, 18.6MB/s]

Extracting files...


Dataset downloaded to: C:\Users\rayal\.cache\kagglehub\datasets\hugodarwood\epirecipes\versions\2
Copied to: ./data/full_format_recipes.json


In [4]:
# Load the full dataset
# Note: Update this path to match where you saved the dataset
with open("./data/full_format_recipes.json") as json_data:
    recipe_data = json.load(json_data)

In [5]:
# Filter the dataset
filtered_data = [
    "Recipe for " + x["title"] + " | " + " ".join(x["directions"])
    for x in recipe_data
    if "title" in x
    and x["title"] is not None
    and "directions" in x
    and x["directions"] is not None
]

In [6]:
# Count the recipes
n_recipes = len(filtered_data)
print(f"{n_recipes} recipes loaded")

20111 recipes loaded


## 2. Tokenise the data

In [8]:
# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


text_data = [pad_punctuation(x) for x in filtered_data]

In [9]:
# Display an example of a recipe
example_data = text_data[9]
example_data

'Recipe for Ham Persillade with Mustard Potato Salad and Mashed Peas | Chop enough parsley leaves to measure 1 tablespoon ; reserve . Chop remaining leaves and stems and simmer with broth and garlic in a small saucepan , covered , 5 minutes . Meanwhile , sprinkle gelatin over water in a medium bowl and let soften 1 minute . Strain broth through a fine - mesh sieve into bowl with gelatin and stir to dissolve . Season with salt and pepper . Set bowl in an ice bath and cool to room temperature , stirring . Toss ham with reserved parsley and divide among jars . Pour gelatin on top and chill until set , at least 1 hour . Whisk together mayonnaise , mustard , vinegar , 1 / 4 teaspoon salt , and 1 / 4 teaspoon pepper in a large bowl . Stir in celery , cornichons , and potatoes . Pulse peas with marjoram , oil , 1 / 2 teaspoon pepper , and 1 / 4 teaspoon salt in a food processor to a coarse mash . Layer peas , then potato salad , over ham . '

In [10]:
# Convert to a Tensorflow Dataset
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [11]:
# Create a vectorisation layer
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [12]:
# Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [13]:
# Display some token:word mappings
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: .
3: ,
4: and
5: to
6: in
7: the
8: with
9: a


In [14]:
# Display the same example converted to ints
example_tokenised = vectorize_layer(example_data)
print(example_tokenised.numpy())

[  26   16  557    1    8  298  335  189    4 1054  494   27  332  228
  235  262    5  594   11  133   22  311    2  332   45  262    4  671
    4   70    8  171    4   81    6    9   65   80    3  121    3   59
   12    2  299    3   88  650   20   39    6    9   29   21    4   67
  529   11  164    2  320  171  102    9  374   13  643  306   25   21
    8  650    4   42    5  931    2   63    8   24    4   33    2  114
   21    6  178  181 1245    4   60    5  140  112    3   48    2  117
  557    8  285  235    4  200  292  980    2  107  650   28   72    4
  108   10  114    3   57  204   11  172    2   73  110  482    3  298
    3  190    3   11   23   32  142   24    3    4   11   23   32  142
   33    6    9   30   21    2   42    6  353    3 3224    3    4  150
    2  437  494    8 1281    3   37    3   11   23   15  142   33    3
    4   11   23   32  142   24    6    9  291  188    5    9  412  572
    2  230  494    3   46  335  189    3   20  557    2    0    0    0
    0 

## 3. Create the Training Set

In [15]:
# Create the training set of recipes and the same text shifted by one word
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

## 4. Build the LSTM <a name="build"></a>

In [16]:
inputs = layers.Input(shape=(None,), dtype="int32")
x = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x = layers.LSTM(N_UNITS, return_sequences=True)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
lstm = models.Model(inputs, outputs)
lstm.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 100)      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 128)      │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 10000)    │     1,290,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,407,248 (9.18 MB)

 Trainable params: 2,407,248 (9.18 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
if LOAD_MODEL:
    lstm = models.load_model("./models/lstm.keras", compile=False)

## 5. Train the LSTM <a name="train"></a>

In [18]:
loss_fn = losses.SparseCategoricalCrossentropy()
lstm.compile("adam", loss_fn)

In [19]:
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y = self.model.predict(x, verbose=0)
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            info.append({"prompt": start_prompt, "word_probs": probs})
            start_tokens.append(sample_token)
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("recipe for", max_tokens=100, temperature=1.0)

In [24]:
# Create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.weights.h5",
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

# Tokenize starting prompt
text_generator = TextGenerator(vocab)

In [25]:
lstm.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, tensorboard_callback, text_generator],
)

Epoch 1/25
629/629 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 5.0463
generated text:
recipe for steamed cobbler with can aside 

629/629 ━━━━━━━━━━━━━━━━━━━━ 670s 1s/step - loss: 4.1824
Epoch 2/25
629/629 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 3.1215
generated text:
recipe for salt with eggs and roasted verbena tequila | baking sheet on large refrigerated . line watercress aside . * unavailable , right yams of ice liquid in a jar . working in moderate heat over moderate heat until well . brown 2 x 8 cups liquid down , then moderately high heat . stir in a sauté 2 minutes , clarified at a separate pot and add the liquid and garlic until tester through like and open the the excess lengths . pour hot on incorporate mixture will combined ) or freezer until golden and reserve green sides

629/629 ━━━━━━━━━━━━━━━━━━━━ 721s 1s/step - loss: 2.9402
Epoch 3/25
629/629 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 2.5580
generated text:
recipe for onion - candied new fudge potatoes cookies | , 1 1 / 2 - 

In [26]:
# Save the final model
lstm.save("./models/lstm.keras")

## 6. Generate text using the LSTM

In [23]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        print(f"\nPROMPT: {i['prompt']}")
        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

## 7. Text Generation with Different Temperatures

I will generate text with two different temperature values (0.5, 1.5)

### Temperature = 0.5 (Lower temperature - more focused, deterministic output)

In [27]:
info = text_generator.generate(
    "recipe for", max_tokens=100, temperature=0.5
)


generated text:
recipe for grilled corn - roasted pepper with lemon and ricotta | place 1 cup hot milk and 1 / 2 cup water in medium bowl . add enough hot water to cover by 2 inches . add water , 1 / 2 cup water , 1 / 2 cup water , 1 / 4 cup water , 1 / 4 cup sugar , and 1 / 4 teaspoon salt and simmer 2 minutes . remove from heat ; let stand at room temperature until mixture is tender , about 25 minutes . whisk in 1 cup water . bring



In [28]:
print_probs(info, vocab)


PROMPT: recipe for
grilled:   	13.079999923706055%
roasted:   	9.739999771118164%
chicken:   	6.28000020980835%
[UNK]:   	5.420000076293945%
chocolate:   	5.21999979019165%
--------


PROMPT: recipe for grilled
pork:   	29.059999465942383%
chicken:   	20.969999313354492%
steak:   	10.569999694824219%
lamb:   	7.929999828338623%
tuna:   	4.159999847412109%
--------


PROMPT: recipe for grilled corn
and:   	44.75%
with:   	44.25%
-:   	4.71999979019165%
bread:   	2.5199999809265137%
,:   	0.8299999833106995%
--------


PROMPT: recipe for grilled corn -
roasted:   	47.060001373291016%
marinated:   	13.760000228881836%
stuffed:   	9.319999694824219%
grilled:   	5.840000152587891%
fried:   	3.450000047683716%
--------


PROMPT: recipe for grilled corn - roasted
red:   	20.75%
pork:   	13.229999542236328%
carrots:   	7.980000019073486%
chicken:   	6.380000114440918%
salmon:   	5.300000190734863%
--------


PROMPT: recipe for grilled corn - roasted pepper
with:   	80.94000244140625%
-:   	6.

### Temperature = 1.5 (Higher temperature - more creative, diverse output)

In [29]:
info = text_generator.generate(
    "recipe for", max_tokens=100, temperature=1.5
)


generated text:
recipe for vinegared stone semolina | place large nonmetal somewhere ( to ensure that coals rub ) . melt butter in a heatproof 10 - large nonstick skillet cover with brush nuts in pea seeds . working with almonds firmly onto paper towels ; push chocolate just to make in cider flakes . working with donatella masher force fingers can separates as dal is hot , leave separate into quarters followed by hand and work the batch through sieve carefully smooth 3 drinks mixing gelato cereal . put matzo - bourbon balls skewer and scoop each cloves so mini hummus



In [30]:
print_probs(info, vocab)


PROMPT: recipe for
grilled:   	0.8600000143051147%
roasted:   	0.7799999713897705%
chicken:   	0.6800000071525574%
[UNK]:   	0.6399999856948853%
chocolate:   	0.6399999856948853%
--------


PROMPT: recipe for vinegared
rice:   	10.430000305175781%
couscous:   	2.059999942779541%
chicken:   	1.6100000143051147%
cucumber:   	0.9700000286102295%
corn:   	0.9300000071525574%
--------


PROMPT: recipe for vinegared stone
fruit:   	13.020000457763672%
|:   	5.320000171661377%
-:   	2.7699999809265137%
[UNK]:   	1.5%
rice:   	1.440000057220459%
--------


PROMPT: recipe for vinegared stone semolina
|:   	39.040000915527344%
and:   	3.119999885559082%
pudding:   	1.940000057220459%
bars:   	1.5800000429153442%
squares:   	1.4900000095367432%
--------


PROMPT: recipe for vinegared stone semolina |
in:   	8.149999618530273%
preheat:   	6.769999980926514%
combine:   	4.269999980926514%
mix:   	4.059999942779541%
place:   	3.7899999618530273%
--------


PROMPT: recipe for vinegared stone semolin

## Summary and Observation

### What I Did

I trained an LSTM neural network on 20,111 recipes from the Epicurious dataset. The model learned to predict the next word in a recipe based on the words before it. Then I generated new recipes using two different temperature settings to see how it affects the output.

### Results

**Temperature = 0.5 (Safe Mode)**
The model produced recipes that actually made sense. It repeated common cooking words like "salt," "pepper," and "minutes" a lot, but the steps were logical and realistic. It's like the model played it safe and stuck to what it knows.

**Temperature = 1.5 (Creative Mode)**
The model got weird and experimental. It mixed ingredients in strange ways - like combining things that don't usually go together. The recipes were fun to read but didn't make much sense. It felt like the model was taking big risks.

### Key Insight

**Temperature is like a "creativity dial":**
- **Low temperature (0.5)** = predictable and safe
- **High temperature (1.5)** = random and creative

This shows that the same model can produce completely different results just by changing one number. It's cool how much control we have over what the AI generates.

### How It Works

The LSTM has 128 units and 10,000 word vocabulary. When generating text:
1. Start with "recipe for"
2. Model predicts next word (picks from 10K options)
3. Add that word to the sequence
4. Repeat until the recipe is done

Temperature works by adjusting probabilities:
- **T < 1**: Makes likely words even more likely → stable output
- **T > 1**: Makes all words equally possible → unpredictable output

---

## AI Usage Note

This assignment used AI (GitHub Copilot) for:
- Code structure and organization (callbacks, TextGenerator class design)
- Summarizing findings and writing explanations

All model training and text generation results were produced independently, and conclusions are based on actual outputs from the trained LSTM.

## References

Goodfellow, I., Bengio, Y., & Courville, A. (2016). *Deep Learning*. MIT Press.

Chollet, F. (2021). Deep Learning with Python (2nd ed.). Manning Publications.